## What is Model Context Protocol

Model Context Protocol (MCP) allows users to easily plug-and-play 3rd party resources into AI systems with an interface that's easy to understand and use by LLMs.

- **Tip** You can find a list of more popular Model Context Protocol (MCP) resources at [punkpeye/awesome-mcp-servers](https://github.com/punkpeye/)

In this example we'll be focusing only on integrating MCP Tools into the [BAML](https://www.boundaryml.com/) structured generation framework.

## Let's pick an MCP tool: Calculator

Let's start by using [githejie/mcp-server-calculator](https://github.com/githejie/mcp-server-calculator) with BAML.

The instructions show how can we "import" the MCP Server (which contains the calculator tool) into our application:
```json
"mcpServers": {
  "calculator": {
    "command": "python",
    "args": ["-m", "mcp_server_calculator"]
  }
}
```

Let's import the MCP Server into our BAML application:

In [1]:
from pydantic_ai.mcp import MCPServerStdio

server_with_calculator_tool = MCPServerStdio(
    command="python",
    args=["-m", "mcp_server_calculator"],
)

Let's check out the available tools. Note, that the server is automatically started using `async with`.

In [2]:
from pprint import pprint

# Server is not running
async with server_with_calculator_tool as server:
    # Server is now started, we can access the list of tools and run them
    tools = await server.list_tools()
# Server is now stopped again

pprint(tools)

[ToolDefinition(name='calculate',
                description='Calculates/evaluates the given expression.',
                parameters_json_schema={'properties': {'expression': {'title': 'Expression',
                                                                      'type': 'string'}},
                                        'required': ['expression'],
                                        'title': 'calculateArguments',
                                        'type': 'object'},
                outer_typed_dict_key=None,
                strict=None)]


Let's try to use the calculator tool manually and verify the resul:

In [15]:
tool_name = "calculate"
arguments = {"expression": "11 * 12 * 13 * 14"}

async with server_with_calculator_tool as server:
    result = await server.call_tool("calculate", arguments)

pprint(result)

CallToolResult(meta=None, content=[TextContent(type='text', text='24024', annotations=None)], isError=False)


In [ ]:
11 * 12 * 13 * 14

24024

# Manually provide the LLM with the calculator tool

Let's create the BAML file:

In [4]:
%cat baml_src/01_use_calculator_tool.baml

class CalculatorToolArguments {
    expression string
}

class UseCalculatorToolOutput {
    tool_name "calculate"
    arguments CalculatorToolArguments
}

function UseCalculatorTool(goal: string) -> UseCalculatorToolOutput {
    client Default
    prompt #"
        Use the calculator to solve: {{ goal }}

        {{ ctx.output_format }}
    "#
}

test usage_example {
    functions [UseCalculatorTool]
    args {
        goal "Multiply all numbers between 10 and 15"
    }
}

Let's test it:

In [5]:
from baml_client.async_client import b

inputs = await b.UseCalculator("Multiply all numbers between 10 and 15")

UseCalculatorToolOutput(tool_name='calculate', arguments=CalculatorToolArguments(expression='11 * 12 * 13 * 14'))

In [13]:
async with server_with_calculator_tool as server:
    result = await server.call_tool(tool_name=inputs.tool_name, arguments={"expression": inputs.arguments.expression})
print(result)

meta=None content=[TextContent(type='text', text='24024', annotations=None)] isError=False


Works great! However, it's tedious to write all this BAML code. Let's automate it.

# Automatically provide the LLM with the calculator tool